In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Verificar e instalar TensorFlow si no está instalado
try:
    import tensorflow as tf
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install tensorflow
    import tensorflow as tf

# ----- Sistema dinámico -----
Gamma = 5
H = 3
t_span = (0, 500)  # Intervalo de tiempo
t_eval = np.linspace(*t_span, 1000)  # Puntos de evaluación

# Sistema de ecuaciones diferenciales
def sistema_ecuaciones(t, vars, Gamma, H):
    x, y, z, w = vars
    dxdt = Gamma * (x - y - 1.35 * x * y - x * y**2)
    dydt = Gamma * (H * x - 3 * y + 1.35 * x * y + x * y**2 + 4 * (w - y))
    dzdt = Gamma * (z - w - 1.35 * z * w - z * w**2)
    dwdt = Gamma * (H * z - 3 * w + 1.35 * z * w + z * w**2 + 2 * 4 * (y - w))
    return [dxdt, dydt, dzdt, dwdt]

# Generar soluciones para entrenamiento
num_samples = 500
alphas_train = np.random.rand(num_samples, 4)
signals_train = []

for alpha in alphas_train:
    sol = solve_ivp(sistema_ecuaciones, t_span, [0, 0, 0.1, 0], args=(Gamma, H), t_eval=t_eval)
    x, y, z, w = sol.y
    ecg_signal = alpha[0] * x + alpha[1] * y + alpha[2] * z + alpha[3] * w
    signals_train.append(ecg_signal)

signals_train = np.array(signals_train)

# ----- Autoencoder -----
input_dim = signals_train.shape[1]  # Número de puntos en la señal
encoding_dim = 16  # Dimensión comprimida

# Capa de entrada
input_signal = tf.keras.Input(shape=(input_dim,))

# Codificador
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_signal)

# Decodificador
decoded = tf.keras.layers.Dense(4, activation='linear')(encoded)

# Modelo autoencoder
autoencoder = tf.keras.Model(input_signal, decoded)

# Compilar el modelo
autoencoder.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
history = autoencoder.fit(signals_train, alphas_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Graficar la pérdida durante el entrenamiento
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Pérdida durante el entrenamiento del Autoencoder')
plt.xlabel('Épocas')
plt.ylabel('Pérdida MSE')
plt.legend()
plt.grid()
plt.show()

# ----- Probar con una nueva señal -----
alpha_test = [0.1, 0.2, 0.3, 0.4]
sol = solve_ivp(sistema_ecuaciones, t_span, [0, 0, 0.1, 0], args=(Gamma, H), t_eval=t_eval)
x, y, z, w = sol.y
ecg_test = alpha_test[0] * x + alpha_test[1] * y + alpha_test[2] * z + alpha_test[3] * w

# Predecir alphas usando el autoencoder
predicted_alpha = autoencoder.predict(ecg_test.reshape(1, -1))[0]

print("Alpha real:", alpha_test)
print("Alpha predicho:", predicted_alpha)
